In [1]:
# Import appropriate modules from the client library.
from googleads import dfp
from collections import defaultdict
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import expr
from pyspark.sql.functions import *
import pandas as pd
import time
sc.stop()
sc =SparkContext()
sqlCtx = SQLContext(sc)


In [2]:
# Campaign details from MARK 
rdd = sc.textFile("gs://ds-url-catag/campaign_analysis/campaign_details.csv") \
    .map(lambda line: line.split(";"))     
first_line = rdd.first()
campaigns_list = rdd.filter(lambda line: line != first_line).toDF(['orders_id','order','audseg_idname','audseg_id','impr','click','category'])


In [3]:
#details of the campaigns - This has to be removed 
campaign_details = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/processed_orders/")

#tagging the campagin list - the campaings that use our model
campaigns_list = campaigns_list.withColumn("To_check",lit(True))

#get the details of the campagins list
cond = [campaign_details.order_id == campaigns_list.orders_id,campaign_details.target_audienceid == campaigns_list.audseg_id]

#getting the detailds of all the campaign list
campaignslisttier1 = campaigns_list.join(campaign_details,cond,'left')

In [4]:
#reading all the campaign details obtained from datapreprocessing and scraping - previous two steps 
filtered_campaigns = sqlCtx.read.parquet("gs://ds-url-catag/campaign_analysis/campaign_details/")
filtered_campaigns = filtered_campaigns.select(col("order_id").alias("orderid"),col("Order_name"),col("IABTier1Categorization"),col("IOEndDate"),col("IOStartDate"))

#get the details of the filtered campaigns
cond = [campaign_details.order_id == filtered_campaigns.orderid]

#getting the details of all the campaign list
filtcamptier1 = filtered_campaigns.join(campaign_details,cond,'left')

#To filter the campaigns before applying model "TO_check !=  True"
filtcamptier2 = filtcamptier1.join(campaigns_list,filtcamptier1.orderid == campaigns_list.orders_id,'left')
filtcamptier2 =filtcamptier2.withColumn("To_check",when(col('To_check') == True,col('To_check')).otherwise(False))
filtcamptier3 = filtcamptier2.where(col('To_check') == False )

In [5]:
#comparing campaigns 
from pyspark.sql.types import BooleanType,StringType
import re
#'DBS',indosat','grab','Google roadblock','uber','opera']

#GOJEK
def regex_filter(x):
    list_campaign = ['uber']
    filter_status = ''
    string_x = x[:5]
    for camp in list_campaign:
        if camp.lower() in string_x.lower():
            filter_status = True
    return filter_status
filter_udf = udf(regex_filter, BooleanType())

#Before application of the model
filtcamptier3 = filtcamptier3.withColumn('Filter_campaigns',filter_udf(filtcamptier3.Order_name))    

In [25]:
#gojek old
go_jek_old = filtcamptier3.where(col('Filter_campaigns') == True)
go_jek_old = go_jek_old.withColumn("filter",when((col('Age_group_1') != "null"),True).otherwise(False))
go_jek_old = go_jek_old.where(col('filter') == True)
go_jek_old = go_jek_old.withColumn("CTR",((col('clicks')/col('impressions'))*100))
old = go_jek_old.where((col('CTR') != 0))


In [14]:
go_jek_old.count()

7

In [24]:
go_jek_old.select(['orderid','Order_name','ad_type_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','Age_group_5','CTR']).show(1300)

+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+------------------+
|   orderid|          Order_name|ad_type_1|Age_group_1|Age_group_2|Age_group_3|Age_group_4|Age_group_5|               CTR|
+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+------------------+
| 407070516|Uber_20160601_201...|     diau|      25-34|      35-44|      18-24|      45-54|      13-17|               0.0|
| 407070516|Uber_20160601_201...|     dnau|      25-34|      35-44|      18-24|      45-54|      13-17|2.2183111399623376|
| 407070516|Uber_20160601_201...|     clau|      25-34|      35-44|      18-24|      45-54|      13-17| 4.913771798997303|
|2174257350|Uber-20171029-A_2...|     dnau|      13-17|      35-44|      25-34|      18-24|       null|0.9187549061107235|
|2193911929|Uber-20171119-A_2...|     dnau|      18-24|      13-17|      25-34|      35-44|       null|0.8875411926557553|
|2188576502|Uber

In [27]:
#gojek new
campaignslisttier1 = campaignslisttier1.withColumn('Filter_campaigns',filter_udf(campaignslisttier1.order))
go_jek_new = campaignslisttier1.where(col('Filter_campaigns') == True)
go_jek_new = go_jek_new.withColumn("filter",when(col('Age_group_1') != "null",True).otherwise(False))
new = go_jek_new.where(col('filter') == True)
new = new.withColumn("CTR",((col('clicks')/col('impressions'))*100))

In [21]:
new.count()

16

In [28]:
new.select(['orders_id','order','ad_type_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','Age_group_5','CTR']).show(go_jek_new.count())

+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+------------------+
| orders_id|               order|ad_type_1|Age_group_1|Age_group_2|Age_group_3|Age_group_4|Age_group_5|               CTR|
+----------+--------------------+---------+-----------+-----------+-----------+-----------+-----------+------------------+
|2197426939|Uber-20171203-A_2...|     dnau|      18-24|      13-17|      25-34|      35-44|       null|0.8015334818760835|
|2210896988|Uber-20171224-A_2...|     dnau|      35-44|      18-24|      25-34|       null|       null|1.0640891946714384|
|2210941604|Uber-20171224-B_2...|     dnau|      35-44|      18-24|      25-34|       null|       null|1.0735466644615392|
|2211228630|Uber-20171224-C_2...|     dnau|      35-44|      18-24|      25-34|       null|       null|0.9098561931045728|
|2214304737|Uber-20171228-B_2...|     dnau|      35-44|      18-24|      25-34|       null|       null|1.0421000904092295|
|2215195916|Uber

In [26]:
#calculating statistics
#no of campaigns
print old.count()

#calculate the difference between timestamps
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd"
timeDiff = (F.unix_timestamp('IOEndDate', format=timeFmt)
            - F.unix_timestamp('IOStartDate', format=timeFmt))
#for old
old = old.withColumn("Duration", (((timeDiff/60)/60/24)))

# #avg no of days 
print old.agg(avg(col('Duration'))).show()

#impressions and clicks 
old = old.withColumn("CTR",((col('clicks')/col('impressions'))*100))

#CTR calculation - check the conditions to filter 
rem_out = old.where(col('CTR').isNotNull())
print rem_out.agg(avg(col('CTR'))).show()


#Average Budget 
print old.agg(avg(col('budget'))).show(1,False)

print old.groupby('target_country').count().show()

6
+-------------+
|avg(Duration)|
+-------------+
|         28.5|
+-------------+

None
+------------------+
|          avg(CTR)|
+------------------+
|1.7968706345979253|
+------------------+

None
+---------------------+
|avg(budget)          |
+---------------------+
|2.9443516314166668E10|
+---------------------+

None
+--------------+-----+
|target_country|count|
+--------------+-----+
|     Singapore|    2|
|    Balikpapan|    2|
|    Yogyakarta|    2|
+--------------+-----+

None


In [29]:
#no of campaigns
print new.count()

#calculate the difference between timestamps
from pyspark.sql import functions as F
timeFmt = "yyyy-MM-dd"
timeDiff = (F.unix_timestamp('End_date', format=timeFmt)
            - F.unix_timestamp('start_date', format=timeFmt))
#for old
new = new.withColumn("Duration", (((timeDiff/60)/60/24)))

#avg no of days 
print new.agg(avg(col('Duration'))).show()

# impressions and clicks 
new = new.withColumn("CTR",((col('clicks')/col('impressions'))*100))

#CTR calculation - check the conditions to filter 
new = new.where(col('CTR') !=0)
print new.agg(avg(col('CTR'))).show()

#Average Budget 
print new.agg(avg(col('budget'))).show(1,False)


print new.groupby('target_country').count().show()

16
+-------------+
|avg(Duration)|
+-------------+
|       7.3125|
+-------------+

None
+------------------+
|          avg(CTR)|
+------------------+
|0.9652474543107983|
+------------------+

None
+-----------+
|avg(budget)|
+-----------+
|8.3125E9   |
+-----------+

None
+--------------+-----+
|target_country|count|
+--------------+-----+
|    Yogyakarta|   16|
+--------------+-----+

None


In [39]:
# go_jek_new.select(['orders_id','order','ad_type_1','Age_group_1','Age_group_1','Age_group_2','Age_group_3','Age_group_4','Age_group_5']).show()